In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os

import torch
import vedo

from models.PCAH import PCAH_Net
from utils.datasetHeads import MeshHeadsDataset

In [2]:
train_heads_files = '/home/fmatzkin/.config/JetBrains/PyCharm2022.2/scratches/test_simulated.txt'
images = open(train_heads_files, 'r').read().splitlines()
im_path = os.path.abspath('/home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/augm')

dataset = MeshHeadsDataset(images, img_path=im_path)

device = 'cuda:0'

In [3]:
interp_factor = 0.85

config = {
    'device': device,
    'Heads': True,
    'Lungs': False,
    'latents': 60,
    'inputsize': 1024,
    'interpolate': True,
    'h': int(512 * interp_factor),
    'w': int(512 * interp_factor),
    'slices': int(233 * interp_factor),
}

pcaNet = PCAH_Net(config.copy()).to(device)
pcaNet.load_state_dict(torch.load('Training/PCAH/bestMSE.pt'))
pcaNet.eval()
print('Model loaded')

Model loaded


In [4]:
print(f"Test images: {len(dataset)}")

# I can use any img since all .faces() are the same
ex_img = os.path.join(
    '../Chest-xray-landmark-dataset/Images',
    'CQ500-CT-12_CT Thin Plain_decimated_1perc_dfm.vtk'
)
out_path = os.path.join(im_path, f'pred_PCAH')  # Predictions subfolder
os.makedirs(out_path, exist_ok=True)

faces = vedo.Mesh(ex_img).faces()

for i, sample in enumerate(dataset):
    test_im = sample['image'].unsqueeze(0)
    out_img = pcaNet(test_im.to(device))
    points = out_img.reshape(-1, 3).detach().cpu().numpy()

    out_fpath = os.path.join(out_path, images[i].replace('.nii.gz', '.stl'))

    restored_mesh = vedo.Mesh([points, faces])
    restored_mesh.write(out_fpath)
    print(f'saved mesh {out_fpath}')
    del test_im, out_img


Test images: 6
File /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/augm/CQ500-CT-39_CT PRE CONTRAST THIN1_sim_decimated_1perc_dfm.vtk not found.
saved mesh /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/augm/pred_PCAH/CQ500-CT-39_CT PRE CONTRAST THIN1_sim.stl
File /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/augm/CQ500-CT-39_CT PRE CONTRAST THIN0_sim_decimated_1perc_dfm.vtk not found.
saved mesh /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/augm/pred_PCAH/CQ500-CT-39_CT PRE CONTRAST THIN0_sim.stl
File /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/augm/CQ500-CT-12_CT Thin Plain1_sim_decimated_1perc_dfm.vtk not found.
saved mesh /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/augm/pred_PCAH/CQ500-CT-12_CT Thin Plain1_sim.stl
File /home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/augm/CQ500-CT-12_CT Thin Plain0_sim_decimated_1perc_dfm.vtk not f

In [12]:
images/sitkImageReaderBase.cxx:97:
sitk::ERROR: The file "

['CQ500-CT-39_CT PRE CONTRAST THIN1_sim.nii.gz ',
 'CQ500-CT-39_CT PRE CONTRAST THIN0_sim.nii.gz ',
 'CQ500-CT-12_CT Thin Plain1_sim.nii.gz ',
 'CQ500-CT-12_CT Thin Plain0_sim.nii.gz ',
 'CQ500-CT-33_CT 4cc sec 150cc D3D on0_sim.nii.gz ',
 'CQ500-CT-33_CT 4cc sec 150cc D3D on1_sim.nii.gz ']

In [ ]:
from headctools.tools.offline_augmentation import simulate_cr_images as sim
sim('/home/fmatzkin/Code/Chest-xray-landmark-dataset/Images/some_of_test/',2,)